In [1]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

  #Clase para representar el nodo como un string
    def __repr__(self):
        rep = 'Node(' + str(self.state)+')\n'
        return rep

  #Clase para compara los estados de los nodos
    def __eq__(self, node):
        return self.state == node.state
  

In [2]:
nodos = [Node('a',None,'x',3), Node('b',None, 'y', 1)]
nodos.sort(key=lambda x:x.path_cost)
#sorted(nodos, key=lambda nodo : nodo.matricula)
print(nodos)

[Node(b)
, Node(a)
]


In [3]:
class Problem(object):
  
  def __init__(self,mapa, initial, goal):
    
    if not isinstance(initial,Node):
      raise TypeError('node type is required for initial')
    if not isinstance(goal,Node):
      raise TypeError('node type is required for initial')

    self.initial = initial
    self.goal = goal
    
  def expand(self, node):
    
    raise NotImplementedError
  
  def evaluar(self,node):
    raise NotImplementedError
  
  def is_Goal(self,node):
    raise NotImplementedError


In [5]:
import json

with open ('Laberinto1.json') as archivo:
  mapa = json.load(archivo)
print(mapa["Mazemap"])


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
class BreadthFirstSearch:
  def __init__(self, problem):
    self.open=[]
    self.close=[]
    self.children=[]
    self.problem=problem

  def run(self):
    self.open.append(self.problem.initial)
    solution=[]
    while self.open :
      actual=self.open.pop(0)
      print('actual',actual)
      if(actual == self.problem.goal):
        while actual:
          action=actual.action
          actual=actual.parent
          solution.append(action)
        solution.reverse()
        return solution

      else:
        self.close.append(actual)
        self.children=self.problem.expand(actual)
        self.clean()
        self.open.extend(self.children)
        #Metodo para ordenar los nodos por el costo
        self.open = self.insert(self.open)
    return solution

  def clean(self):
     for n in self.children[:]:
      for m in self.open:
        if(n==m and n.path_cost < m.path_cost): #se realiza la comparacion
          self.open.remove(m) #se remueve m de open y remover de cerrados
        if(n==m and n.path_cost > m.path_cost):#se realiza la comparacion
          self.children.remove(n)#se remueve n 
      for n in self.children[:]:
        for m in self.close:  
          if(n==m):
            self.children.remove(n)
  
  def insert (self,lista):
    return sorted(lista, key=lambda x:x.path_cost )



In [7]:
import copy

class Laberinto(Problem):
  def __init__(self, mapa, initial, goal):
    Problem.__init__(self, mapa, initial, goal)
    
  def expand(self, node):
    children = []
    
    l=self.left(node)
    if l is not None:
      children.append(l)

    r=self.right(node)
    if r is not None:
      children.append(r)
    
    u=self.up(node)
    if u is not None:
      children.append(u)

    d=self.down(node)
    if d is not None:
      children.append(d)

    return children

  def left(self,node):
    state=node.state
    i,j=state
    if(i>1 and j>1 and j<2):
      newstate=copy.deepcopy(state)
      valor=newstate[i][j-1]
      newstate[i][j-1]=0
      newstate[i][j]=valor
      costo = self.evaluacion(node)
      newnode=Node(newstate,node,'left',costo)
      print("Nodo", newnode.action,"peso",newnode.path_cost)
      return newnode
    else:
      return None

  def right(self,node):
    state=node.state
    i,j=state
    if(i>1 and j>1 and j>0):
      newstate=copy.deepcopy(state)
      valor=newstate[i][j-1]
      newstate[i][j+1]=0
      newstate[i][j]=valor
      costo = self.evaluacion(node)
      newnode=Node(newstate,node,'right')
      print("Nodo", newnode.action,"peso",newnode.path_cost)
      return newnode
    else:
      return None

  def up(self,node):
    state=node.state
    i,j=state
    if(i>-1 and j>-1 and i<2):
      newstate=copy.deepcopy(state)
      valor=newstate[i+1][j]
      newstate[i+1][j]=0
      newstate[i][j]=valor
      costo = self.evaluacion(node)
      newnode=Node(newstate,node,'up')
      print("Nodo", newnode.action,"peso",newnode.path_cost)
    else:
      return None

  def down(self,node):
    state=node.state
    i,j=state
    if(i>1 and j>1 and i>0):
      newstate=copy.deepcopy(state)
      valor=newstate[i-1][j]
      newstate[i-1][j]=0
      newstate[i][j]=valor
      costo = self.evaluacion(node)
      newnode=Node(newstate,node,'dopwn')
      print("Nodo", newnode.action,"peso",newnode.path_cost)
      return newnode
    else:
      return None

    def evaluar(self, node):
      x0 = node.state[0]
      y0 = node.state[1]
      xf = self.goal.state[0]
      yf = self.goal.state[1]

      f_h = round(node.depth + abs((xf-x0)+ abs(yf-y0)))
    return f_h

    def findgap(self, state):
        for i, row in enumerate(state):
          for j, col in enumerate(row):
            if col==0 :
              return i, j
        return -1,-1

In [8]:
import json
def main():
  with open ('Laberinto1.json') as archivo:
    mapa = json.load(archivo)

    initial = Node(mapa["Start"])
    goal = Node(mapa["Goal"])
    map = Laberinto(mapa["Mazemap"], initial,goal)
    bfs=BreadthFirstSearch(map)
    solution=bfs.run()
    print(solution)

if __name__ == "__main__":
    main()

actual Node([12, 0])

[]
